[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hemanth1403/IIITH-Student-training-program-AIML/blob/main/AIML_Module_3_Lab_3_Using_KNN_for_Text_Classification_ipynb.ipynb)

### MODULE: CLASSIFICATION-1
### LAB-3 : Using KNN for Text Classification



---

## **Section 1: Understanding NLP tools**

In this lab we will be using KNN on a real world NLP application i.e. is text classification. But first look at some NLP techniques for text classification and tools that we use when we want to use python for NLP.

## Section 1.2: Data Cleaning and Preprocessing step

Raw text must be processed and converted into a form so that it is suitable to use with various machine-learning algorithms.  
In case of text, there are lots of things that need to be taken into account.  


1.   Removing numbers from the text
2.   Handling capitalization and punctuation.
3.   Stemming and Lemmatizing text.  

And most importantly, one can't just use words or images directly in algorithms; they need to be converted into vectors- a form that algorithms can understand.



### **NLTK**
NLTK (or Natural Language Tool Kit) is a commonly used library for processing text. We will use this tool in this lab. Lets first install it.


In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /Users/hemu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/hemu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hemu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/hemu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/hemu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import re
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

def cleanText(text, lemmatize, stemmer):
    """Method for cleaning text from train and test data. Removes numbers, punctuation, and capitalization. Stems or lemmatizes text."""

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^A-Za-z]", " ", text)
    text = text.lower()


    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def get_tag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_result = []
        tokens = word_tokenize(text)  # Generate list of tokens
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
            except:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_result

    if stemmer:
        text_result = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('english')
        for t in tokens:
            text_result.append(snowball_stemmer.stem(t))
        return text_result

In [6]:
sample_text = "Troubling"
sample_text_result = cleanText(sample_text, lemmatize=False, stemmer=True)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text)
print(sample_text_result)
sample_text_result = cleanText(sample_text, lemmatize=True, stemmer=False)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text_result)

Troubling
troubl
trouble


## Section 1.2: BAG OF WORDS

A bag-of-words model, or BoW for short, is a way of extracting features from text for use in modeling, such as with machine learning algorithms.

The approach is very simple and flexible, and can be used in many ways for extracting features from documents.

A bag-of-words is a representation of text that describes the occurrence of words within a document.
It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

In [7]:
5*12

60

In [8]:
# Functions to convert document(s) to a list of words, with the option of removing stopwords. Returns document-term matrix.

def createBagOfWords(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = CountVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer = CountVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    bag_of_words_train = vectorizer.fit_transform(clean_train).toarray()
    bag_of_words_test = vectorizer.transform(clean_test).toarray()
    return bag_of_words_train, bag_of_words_test


## Section 1.3: TF-IDF
TF-IDF technique is used to find meaning of sentences consisting of words and cancels out the incapabilities of Bag of Words technique which is good for text classification or for helping a machine read words in numbers.

The number of times a term occurs in a document is called its Term frequency (TF).

 Document frequency is the number of documents in which the word is present.  Inverse DF (IDF) is the inverse of the document frequency which measures the informativeness of term *t*.




In [9]:
def createTFIDF(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = TfidfVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    tfidf_train = vectorizer.fit_transform(clean_train).toarray()
    tfidf_test = vectorizer.transform(clean_test).toarray()
    return tfidf_train, tfidf_test

# **Section 2: UNDERSTANDING THE DATA : A REVIEWS DATASET**

Sentiment analysis is the interpretation and classification of emotions (such as positive, negative and neutral) within text data using text analysis techniques.  
Given below is a dataset consisting of reviews along with sentiment class (positive or negative).

In [ ]:
# Upload the Reviews CSV file that has been shared with you.
# Run this cell, click on the 'Choose files' button and upload the file.

# from google.colab import files
# uploaded = files.upload()

Saving reviews.csv to reviews.csv


In [10]:
import pandas as pd
df = pd.read_csv('./reviews.csv')

In [11]:
df = df.dropna()

In [12]:
df

,sentence,sentiment
0,Not sure who was more lost - the flat characte...,0
1,Attempting artiness with black & white and cle...,0
2,Very little music or anything to speak of.,0
3,The best scene in the movie was when Gerardo i...,1
4,"The rest of the movie lacks art, charm, meanin...",0
...,...,...
994,I just got bored watching Jessice Lange take h...,0
995,"Unfortunately, any virtue in this film's produ...",0
996,"In a word, it is embarrassing.",0
997,Exceptionally bad!,0


In [14]:
df.to_csv('./reviews.csv', index=False)

# **Section 3: KNN MODEL**

Given below are two KNN models; in the first case we are using Bag-of-Words and in the second case we are using TF-IDF.
Note the different metrics and parameters used in each.

In [47]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 1: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('./reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    
    metric = ["minkowski", "manhattan", "chebyshev", "hamming", "cosine", "euclidean"]
    for _ in metric:
        knn11 = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric=_, metric_params=None, n_jobs=1)
        knn11.fit(X_train, y_train)
        predicted11 = knn11.predict(X_test)
        acc11 = metrics.accuracy_score(y_test, predicted11)
        print('KNN with BOW accuracy (N_neighbours = 5, distance = uniform, metrics = ',_,') = ' + str(acc11 * 100) + '%')
        scores11 = cross_val_score(knn11, X_train, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores11.mean()))
        print(scores11)
        print('\n')
        
        knn12 = neighbors.KNeighborsClassifier(n_neighbors=7, weights='distance', algorithm='auto', leaf_size=30, p=2, metric=_, metric_params=None, n_jobs=1)
        knn12.fit(X_train, y_train)
        predicted12 = knn12.predict(X_test)
        acc12 = metrics.accuracy_score(y_test, predicted12)
        print('KNN with BOW accuracy (N_neighbours = 7, distance = distance, metrics = ',_,') = ' + str(acc12 * 100) + '%')
        scores12 = cross_val_score(knn12, X_train, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores12.mean()))
        print(scores12)
        print('\n')
    return predicted11, y_test

#     knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)
#     knn.fit(X_train, y_train)
#     predicted = knn.predict(X_test)
#     acc = metrics.accuracy_score(y_test, predicted)
#     print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

#     scores = cross_val_score(knn, X_train, y_train, cv=3)
#     print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
#     print(scores)
#     print('\n')
#     return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"],
                                                        test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)

    metric = ["minkowski", "hamming", "euclidean","cosine"]
    for _ in metric:
        knn11 = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric=_, metric_params=None, n_jobs=1)
        knn11.fit(X_train, y_train)
        predicted11 = knn11.predict(X_test)
        acc11 = metrics.accuracy_score(y_test, predicted11)
        print('KNN with TFIDF accuracy (N_neighbours = 5, distance = uniform, metrics = ',_,') = ' + str(acc11 * 100) + '%')
        scores11 = cross_val_score(knn11, X_train, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores11.mean()))
        print(scores11)
        print('\n')
        
        knn12 = neighbors.KNeighborsClassifier(n_neighbors=7, weights='distance', algorithm='auto', leaf_size=30, p=2, metric=_, metric_params=None, n_jobs=1)
        knn12.fit(X_train, y_train)
        predicted12 = knn12.predict(X_test)
        acc12 = metrics.accuracy_score(y_test, predicted12)
        print('KNN with TFIDF accuracy (N_neighbours = 7, distance = distance, metrics = ',_,') = ' + str(acc12 * 100) + '%')
        scores12 = cross_val_score(knn12, X_train, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores12.mean()))
        print(scores12)
        print('\n')
    return predicted11, y_test

#     knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2,
#                                          metric='cosine', metric_params=None, n_jobs=1)

#     knn.fit(X_train, y_train)
#     predicted = knn.predict(X_test)
#     acc = metrics.accuracy_score(y_test, predicted)
#     print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

#     scores = cross_val_score(knn, X_train, y_train, cv=3)
#     print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
#     print(scores)
#     return predicted, y_test

Note: Cross-validation will be discussed in detail in the upcoming lab session.

In [48]:
## KNN accuracy after using BoW
predicted, y_test = bow_knn()

/var/folders/1t/x8g_6_2j4r35ynvl5w4n0qqh0000gn/T/ipykernel_15491/34442907.py:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BOW accuracy (N_neighbours = 5, distance = uniform, metrics =  minkowski ) = 62.30366492146597%
Cross Validation Accuracy: 0.62
[0.60784314 0.58431373 0.66141732]


KNN with BOW accuracy (N_neighbours = 7, distance = distance, metrics =  minkowski ) = 65.96858638743456%
Cross Validation Accuracy: 0.62
[0.60784314 0.58823529 0.67716535]




/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


KNN with BOW accuracy (N_neighbours = 5, distance = uniform, metrics =  manhattan ) = 62.30366492146597%


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


...

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.61
[0.63137255 0.6        0.60629921]


KNN with BOW accuracy (N_neighbours = 5, distance = uniform, metrics =  cosine ) = 69.63350785340315%
Cross Validation Accuracy: 0.71
[0.70588235 0.69803922 0.73622047]


KNN with BOW accuracy (N_neighbours = 7, distance = distance, metrics =  cosine ) = 70.15706806282722%
Cross Validation Accuracy: 0.71
[0.68627451 0.69411765 0.73622047]




/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


KNN with BOW accuracy (N_neighbours = 5, distance = uniform, metrics =  euclidean ) = 62.30366492146597%
Cross Validation Accuracy: 0.62
[0.60784314 0.58431373 0.66141732]


KNN with BOW accuracy (N_neighbours = 7, distance = distance, metrics =  euclidean ) = 65.96858638743456%
Cross Validation Accuracy: 0.62
[0.60784314 0.58823529 0.67716535]




/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


In [49]:
## KNN accuracy after using TFIDF
predicted, y_test = tfidf_knn()

/var/folders/1t/x8g_6_2j4r35ynvl5w4n0qqh0000gn/T/ipykernel_15491/34442907.py:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TFIDF accuracy (N_neighbours = 5, distance = uniform, metrics =  minkowski ) = 68.58638743455498%
Cross Validation Accuracy: 0.73
[0.73333333 0.74117647 0.70472441]


KNN with TFIDF accuracy (N_neighbours = 7, distance = distance, metrics =  minkowski ) = 71.72774869109948%
Cross Validation Accuracy: 0.74
[0.70980392 0.75686275 0.7519685 ]


KNN with TFIDF accuracy (N_neighbours = 5, distance = uniform, metrics =  hamming ) = 58.1151832460733%


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.54
[0.52941176 0.51764706 0.57086614]


KNN with TFIDF accuracy (N_neighbours = 7, distance = distance, metrics =  hamming ) = 58.1151832460733%


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.54
[0.5372549  0.53333333 0.56299213]


KNN with TFIDF accuracy (N_neighbours = 5, distance = uniform, metrics =  euclidean ) = 68.58638743455498%
Cross Validation Accuracy: 0.73
[0.73333333 0.74117647 0.70472441]


KNN with TFIDF accuracy (N_neighbours = 7, distance = distance, metrics =  euclidean ) = 71.72774869109948%
Cross Validation Accuracy: 0.74
[0.70980392 0.75686275 0.7519685 ]


KNN with TFIDF accuracy (N_neighbours = 5, distance = uniform, metrics =  cosine ) = 70.68062827225131%
Cross Validation Accuracy: 0.73
[0.7254902  0.7372549  0.72440945]


KNN with TFIDF accuracy (N_neighbours = 7, distance = distance, metrics =  cosine ) = 70.15706806282722%


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.75
[0.70588235 0.74901961 0.79133858]




# Section 4: SPAM TEXT DATASET
Now let's use what we've learnt to classify texts as spam or not spam.

In [ ]:
# Upload the spam text data CSV file that has been shared with you. You can also download the file from https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
# Run this cell, click on the 'Choose files' button and upload the file.
# from google.colab import files
# uploaded = files.upload()

Saving spam.csv to spam.csv


In [51]:
import pandas as pd
df = pd.read_csv('./spam.csv')
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ã¼ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [52]:
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

In [53]:
df.head(5)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [54]:
len(df)

5572

In [58]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 2: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

    metric = ["minkowski", "manhattan", "chebyshev", "hamming", "cosine", "euclidean"]
    for _ in metric:
        knn11 = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric=_, metric_params=None, n_jobs=1)
        knn11.fit(X_train, y_train)
        predicted11 = knn11.predict(X_test)
        acc11 = metrics.accuracy_score(y_test, predicted11)
        print('KNN with BOW accuracy (N_neighbours = 5, distance = uniform, metrics = ',_,') = ' + str(acc11 * 100) + '%')
        scores11 = cross_val_score(knn11, X_train, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores11.mean()))
        print(scores11)
        print('\n')
        
        knn12 = neighbors.KNeighborsClassifier(n_neighbors=7, weights='distance', algorithm='auto', leaf_size=30, p=2, metric=_, metric_params=None, n_jobs=1)
        knn12.fit(X_train, y_train)
        predicted12 = knn12.predict(X_test)
        acc12 = metrics.accuracy_score(y_test, predicted12)
        print('KNN with BOW accuracy (N_neighbours = 7, distance = distance, metrics = ',_,') = ' + str(acc12 * 100) + '%')
        scores12 = cross_val_score(knn12, X_train, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores12.mean()))
        print(scores12)
        print('\n')
    return predicted11, y_test

    # knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    # knn.fit(X_train, y_train)
    # predicted = knn.predict(X_test)
    # acc = metrics.accuracy_score(y_test, predicted)
    # print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    # scores = cross_val_score(knn, X_train, y_train, cv=3)
    # print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    # print(scores)
    # print('\n')
    # return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)

    metric = ["minkowski", "hamming", "euclidean","cosine"]
    for _ in metric:
        knn11 = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric=_, metric_params=None, n_jobs=1)
        knn11.fit(X_train, y_train)
        predicted11 = knn11.predict(X_test)
        acc11 = metrics.accuracy_score(y_test, predicted11)
        print('KNN with TFIDF accuracy (N_neighbours = 5, distance = uniform, metrics = ',_,') = ' + str(acc11 * 100) + '%')
        scores11 = cross_val_score(knn11, X_train, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores11.mean()))
        print(scores11)
        print('\n')
        
        knn12 = neighbors.KNeighborsClassifier(n_neighbors=7, weights='distance', algorithm='auto', leaf_size=30, p=2, metric=_, metric_params=None, n_jobs=1)
        knn12.fit(X_train, y_train)
        predicted12 = knn12.predict(X_test)
        acc12 = metrics.accuracy_score(y_test, predicted12)
        print('KNN with TFIDF accuracy (N_neighbours = 7, distance = distance, metrics = ',_,') = ' + str(acc12 * 100) + '%')
        scores12 = cross_val_score(knn12, X_train, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores12.mean()))
        print(scores12)
        print('\n')
    return predicted11, y_test

    # knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2, metric='cosine', metric_params=None, n_jobs=1)

    # knn.fit(X_train, y_train)
    # predicted = knn.predict(X_test)
    # acc = metrics.accuracy_score(y_test, predicted)
    # print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    # scores = cross_val_score(knn, X_train, y_train, cv=3)
    # print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    # print(scores)
    # return predicted, y_test

In [59]:
# This cell may take some time to run
predicted, y_test = bow_knn()

/var/folders/1t/x8g_6_2j4r35ynvl5w4n0qqh0000gn/T/ipykernel_15491/34442907.py:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BOW accuracy (N_neighbours = 5, distance = uniform, metrics =  minkowski ) = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]


KNN with BOW accuracy (N_neighbours = 7, distance = distance, metrics =  minkowski ) = 93.09417040358744%
Cross Validation Accuracy: 0.92
[0.92664872 0.9179004  0.92659933]


KNN with BOW accuracy (N_neighbours = 5, distance = uniform, metrics =  manhattan ) = 91.74887892376682%
Cross Validation Accuracy: 0.90
[0.90309556 0.89771198 0.9030303 ]


KNN with BOW accuracy (N_neighbours = 7, distance = distance, metrics =  manhattan ) = 93.45291479820628%
Cross Validation Accuracy: 0.93
[0.93001346 0.92059219 0.92929293]


KNN with BOW accuracy (N_neighbours = 5, distance = uniform, metrics =  chebyshev ) = 90.94170403587444%
Cross Validation Accuracy: 0.90
[0.90982503 0.88963661 0.9023569 ]


KNN with BOW accuracy (N_neighbours = 7, distance = distance, metrics =  chebyshev ) = 93.27354260089686%
Cross Validation Accur

In [60]:
# This cell may take some time to run
predicted, y_test = tfidf_knn()

/var/folders/1t/x8g_6_2j4r35ynvl5w4n0qqh0000gn/T/ipykernel_15491/34442907.py:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TFIDF accuracy (N_neighbours = 5, distance = uniform, metrics =  minkowski ) = 92.19730941704036%
Cross Validation Accuracy: 0.90
[0.90040377 0.89771198 0.9010101 ]


KNN with TFIDF accuracy (N_neighbours = 7, distance = distance, metrics =  minkowski ) = 93.36322869955157%
Cross Validation Accuracy: 0.92
[0.92261104 0.91386272 0.91919192]


KNN with TFIDF accuracy (N_neighbours = 5, distance = uniform, metrics =  hamming ) = 87.71300448430493%
Cross Validation Accuracy: 0.87
[0.87146703 0.86810229 0.87003367]


KNN with TFIDF accuracy (N_neighbours = 7, distance = distance, metrics =  hamming ) = 90.22421524663677%
Cross Validation Accuracy: 0.90
[0.90780619 0.90040377 0.9003367 ]


KNN with TFIDF accuracy (N_neighbours = 5, distance = uniform, metrics =  euclidean ) = 92.19730941704036%
Cross Validation Accuracy: 0.90
[0.90040377 0.89771198 0.9010101 ]


KNN with TFIDF accuracy (N_neighbours = 7, distance = distance, metrics =  euclidean ) = 93.36322869955157%
Cross Validati

### Questions to Think About and Answer
1. Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
2. Can you think of techniques that are better than both BoW and TF-IDF ?
3. Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.

1. A. The TF-IDF (Term Frequency-Inverse Document Frequency) approach generally results in better accuracy than the Bag-of-Words (BoW) model due to the following reasons:<br>
<br>
a. Term Weighting: In the BoW model, all words are treated equally regardless of their importance in the document. However, TF-IDF assigns a weight to each word which signifies the importance of the word in the document. Words that are frequently appearing in a document but not across documents will have a high TF-IDF score, thus helping with the problem of word importance.<br>
<br>
b. Handling of Rare Words: Rare words will have a higher score in the TF-IDF model, which can help in cases like spam detection where certain rare words might be of high importance.

2. A. Yes, there are several techniques that are often used in Natural Language Processing tasks that can outperform both Bag-of-Words (BoW) and TF-IDF:<br>
<br>
a. Word Embeddings: Word embeddings such as Word2Vec, GloVe, and FastText provide a dense vector representation of words where the semantic similarity between words is captured in the vector space. These models are trained to understand the context in which words appear, so words with similar meanings will have vectors that are close together in the vector space.<br>
<br>
b. Doc2Vec: An extension of Word2Vec, Doc2Vec considers the document level context to generate vectors. It’s useful when we don’t just care about single words, but their context or semantics at the document level.

3. A. Stemming:<br>
<br>
* Pros:<br>
a. It’s a simpler and faster method because it operates on a single word without understanding the context, and that makes it computationally less intensive.<br>
b. It’s useful when the exact word form doesn’t matter as much as the root of the word.<br>
<br>
* Cons:<br>
a. Stemming can often create non-existent words, as it applies a set of language-specific rules to chop off the ends of words. <br>
b. It may not be as precise as lemmatization because it simply removes the last few characters of a word, which can lead to loss of meaning.<br>
<br>
Lemmatization:<br>
<br>
* Pros:<br>
a. It reduces words to their base or root form, known as a lemma, which is linguistically correct.  <br>
b. It understands the context of words in text, which leads to more accurate results. <br>
<br>
* Cons:<br>
a. It’s computationally more intensive than stemming, as it involves looking up the lemma in a dictionary and part-of-speech tagging.<br>
b. It takes more time than stemming because it needs to find a meaningful representation of the word.<br>

### Useful Resources for further reading
1. Stemming and Lemmatization: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
2. TF-IDF and BoW : https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/
3. TF-IDF: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
